## Example:
- convex mip solution to knapsack
- determine optimal product quantities to maximize profits
- suggest re-order quantities based on current inventory and demand

In [2]:
import numpy as np
import pandas as pd
from time import time
import scipy.stats as stats

from IPython.display import display # Allows the use of display() for DataFrames

# Pretty display for notebooks
%matplotlib inline

###########################################
# Suppress matplotlib user warnings
# Necessary for newer version of matplotlib
import warnings
warnings.filterwarnings("ignore", category = UserWarning, module = "matplotlib")
#
# Display inline matplotlib plots with IPython
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
###########################################

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns

import numpy as np
import cvxpy

from collections import namedtuple


In [3]:
# product sales data

sales_info=[{'productid':1,'avg_sales':24,'unit_cost':2,'unit_profit':1,'current_inventory':7,'min_inventory':1},
{'productid':2,'avg_sales':14,'unit_cost':1.6,'unit_profit':4,'current_inventory':10,'min_inventory':1},
{'productid':3,'avg_sales':6,'unit_cost':4,'unit_profit':2,'current_inventory':5,'min_inventory':2},
{'productid':4,'avg_sales':8,'unit_cost':3,'unit_profit':1,'current_inventory':15,'min_inventory':3},
{'productid':5,'avg_sales':12,'unit_cost':1,'unit_profit':4,'current_inventory':4,'min_inventory':1}]

sales_df=pd.DataFrame(sales_info)

sales_df.head()

,avg_sales,current_inventory,min_inventory,productid,unit_cost,unit_profit
0,24,7,1,1,2.0,1
1,14,10,1,2,1.6,4
2,6,5,2,3,4.0,2
3,8,15,3,4,3.0,1
4,12,4,1,5,1.0,4


In [4]:
productids=sales_df['productid'].unique().tolist()

product_info_dict={}

for i in productids:
    
    product_info_dict[i]=[1 for _ in range(int(sales_df[sales_df['productid'] == i]['avg_sales'].iloc[0]))]
    
    
    
    

In [5]:

# Create weights and utilities using NumPy vectorized operations
weights = np.concatenate([
    np.arange(0, len(product_info_dict[i])+1) * sales_df[sales_df['productid'] == i]['unit_cost'].iloc[0]
    for i in product_info_dict.keys()
])

utilities = np.concatenate([
    np.arange(0, len(product_info_dict[i])+1) * sales_df[sales_df['productid'] == i]['unit_profit'].iloc[0]
    for i in product_info_dict.keys()
])



In [6]:
# Create quantities matrix using NumPy
# pad each row with zeros the length of previous products possible quantities
# append zeros to beginning of row then add zeros to end of row 
# following range of possible quantities

quantities_list = [
    np.concatenate([
        np.arange(0, len(product_info_dict[productids[i]])+1),
        np.zeros(len(weights) - (len(product_info_dict[productids[i]])+1))
    ]) if i==0 else 
    np.concatenate([
        np.zeros((np.sum([len(product_info_dict[productids[j]])+1 for j in range(len(productids)) if j<i]))),
        np.arange(0, len(product_info_dict[productids[i]])+1),
        np.zeros(len(weights)-((np.sum([len(product_info_dict[productids[j]])+1 for j in range(len(productids)) if j<i]))\
                               +(len(product_info_dict[productids[i]])+1)))
    ])
    for i in range(len(productids))
]

quantities = np.vstack(quantities_list)

quantities

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [7]:
# The variable we are solving for
selection = cvxpy.Variable(len(weights), boolean=True)

# budget/operating capital
P=400

# space constraint/total qty of products can carry
C=200

# Budget constraint
weight_constraint = weights * selection <= P

# capacity constraint, total quantity of all products store can carry
capacity_constraint = sum(quantities * selection) <= C

# minimum inventory level per product constraints
min_constraints=[]

for i in range(int(quantities.shape[0])):
    
    
    min_constraints+=[
      quantities[i] * selection >= sales_df[sales_df['productid']==list(product_info_dict.keys())[i]]\
['min_inventory'].values[0]  
    ]

    
# all constraints

constraints=[weight_constraint,capacity_constraint]+min_constraints

# Our total utility is the sum of the item utilities (i.e. profit)
total_utility = utilities * selection

# We tell cvxpy that we want to maximize total utility 
# subject to constraints
knapsack_problem = cvxpy.Problem(cvxpy.Maximize(total_utility), constraints)

# Solving the problem
# returns maximum profit and optimal quantities of each product
knapsack_problem.solve(solver=cvxpy.GLPK_MI)




762.0

In [8]:
# recommended quantities of products

quantities.dot(selection.value.T)

array([  1., 105.,  13.,   3.,  78.])

In [9]:
# reorder suggestion for weekly reordering

reorder_window=7

optimal_df=sales_df.copy()

optimal_df['optimal_inventory']=quantities.dot(selection.value.T)\
.tolist()

optimal_df['projected_inventory']=optimal_df['current_inventory']\
-reorder_window*optimal_df['avg_sales']

optimal_df['reorder_qty']=optimal_df['optimal_inventory']\
-optimal_df['projected_inventory']

column_order=['productid',
              'min_inventory',
              'avg_sales',
              'unit_cost', 
              'unit_profit', 
              'current_inventory',
              'optimal_inventory', 
              'projected_inventory',
       'reorder_qty']

optimal_df[column_order].head()

,productid,min_inventory,avg_sales,unit_cost,unit_profit,current_inventory,optimal_inventory,projected_inventory,reorder_qty
0,1,1,24,2.0,1,7,1.0,-161,162.0
1,2,1,14,1.6,4,10,105.0,-88,193.0
2,3,2,6,4.0,2,5,13.0,-37,50.0
3,4,3,8,3.0,1,15,3.0,-41,44.0
4,5,1,12,1.0,4,4,78.0,-80,158.0
